In [1]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from scipy.special import ker,kei

In [2]:
dstide = xr.open_dataset('utide_results.nc')

In [3]:
zi = np.arange(0,len(dstide['height'])-7)
m2i, = np.where(dstide['constituent'] == 'M2')

In [4]:
plt.figure()
plt.plot(dstide['Lsmaj'][zi,m2i],dstide['height'][zi],'b')
plt.plot(dstide['Lsmaj'][zi,m2i]
         +dstide['Lsmaj_ci'][zi,m2i],dstide['height'][zi],'c')
plt.plot(dstide['Lsmaj'][zi,m2i]
         -dstide['Lsmaj_ci'][zi,m2i],dstide['height'][zi],'c')
plt.plot(dstide['Lsmin'][zi,m2i],dstide['height'][zi],'r')
plt.plot(dstide['Lsmin'][zi,m2i]
         +dstide['Lsmin_ci'][zi,m2i],dstide['height'][zi],'m')
plt.plot(dstide['Lsmin'][zi,m2i]
         -dstide['Lsmin_ci'][zi,m2i],dstide['height'][zi],'m')
plt.ylim([0,np.max(dstide['height'][zi])+0.5])

FigureCanvasNbAgg()

(0, <xarray.DataArray 'height' ()>
 array(18.0972))

In [5]:
plt.figure()
plt.plot(dstide['Lsmin'][zi,m2i]/dstide['Lsmaj'][zi,m2i],dstide['height'][zi])
plt.title('minor/major axis ratio')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'minor/major axis ratio')

In [191]:
plt.figure()
plt.plot(dstide['theta'][zi,m2i],dstide['height'][zi],'b')
plt.plot(dstide['theta'][zi,m2i]+dstide['theta_ci'][zi,m2i],
         dstide['height'][zi],'c')
plt.plot(dstide['theta'][zi,m2i]-dstide['theta_ci'][zi,m2i],
         dstide['height'][zi],'c')
plt.title('orientation, $\\theta_q$ (deg)')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'orientation, $\\theta_q$ (deg)')

In [194]:
plt.figure()
plt.plot(dstide['g'][zi,m2i],dstide['height'][zi],'b')
plt.plot(dstide['g'][zi,m2i]+dstide['g_ci'][zi,m2i],
         dstide['height'][zi],'c')
plt.plot(dstide['g'][zi,m2i]-dstide['g_ci'][zi,m2i],
         dstide['height'][zi],'c')
plt.title('phase lag, $g_q$ (deg)')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'phase lag, $g_q$ (deg)')

In [8]:
dstide

<xarray.Dataset>
Dimensions:      (constituent: 59, height: 23)
Coordinates:
  * height       (height) float64 2.597 3.597 4.597 5.597 ... 22.6 23.6 24.6
  * constituent  (constituent) object 'SSA' 'MSM' 'MM' ... 'MSK6' '3MK7' 'M8'
Data variables:
    Lsmaj        (height, constituent) float64 ...
    Lsmin        (height, constituent) float64 ...
    theta        (height, constituent) float64 ...
    g            (height, constituent) float64 ...
    SNR          (height, constituent) float64 ...
    PE           (height, constituent) float64 ...
    Lsmaj_ci     (height, constituent) float64 ...
    Lsmin_ci     (height, constituent) float64 ...
    theta_ci     (height, constituent) float64 ...
    g_ci         (height, constituent) float64 ...
    freq         (constituent) float64 ...
Attributes:
    reftime:  736813.8008534722

| Parameter                    | Soulsby (theory) | Codiga (UTide)           |
|------------------------------|------------------|--------------------------|
| major axis                   | $U_a$            | $L^{smaj}_q$             |
| minor axis                   | $U_b$            | $L^{smin}_q$             |
| complex anticlockwise vector | $R_+$            | $a^+_q$                  |
| complex clockwise vector     | $R_-$            | $a^-_q$                  |
| anticlockwise amplitude      | $|R_+|$          | $A^+_q$                  |
| clockwise amplitude          | $|R_-|$          | $A^-_q$                  |
| orientation angle            | $\Phi$           | $\theta_q$               |
| phase lag                    | $\phi$           | $g_q$                    |
| anticlockwise phase          | $\phi_+$         | $\epsilon^+_q$, -$g^+_q$ |
| clockwise phase              | $\phi_-$         | $\epsilon^-_q$, $g^-_q$  |

In [217]:
i = 1j
ap = 0.5*(dstide['Lsmaj']+dstide['Lsmin'])*np.exp(i*(dstide['theta']-dstide['g'])*np.pi/180)
am = 0.5*(dstide['Lsmaj']-dstide['Lsmin'])*np.exp(i*(dstide['theta']+dstide['g'])*np.pi/180)
Ap = np.abs(ap)
Am = np.abs(am)
epsilonp = np.angle(ap)*180/np.pi
epsilonm = np.angle(am)*180/np.pi

Test calculations (makes sure relationships in Equation 9 of Codiga are satisfied).
All tests should be True.

In [10]:
print(np.sum(~np.isclose(dstide['Lsmaj'],Ap + Am)) == 0)
print(np.sum(~np.isclose(dstide['Lsmin'],Ap - Am)) == 0)
print(np.sum(~(np.isclose(dstide['g'],-epsilonp + dstide['theta']) |
            np.isclose(dstide['g'], -epsilonp + dstide['theta'] + 360))) == 0)

True
True
True


In [11]:
z = np.arange(0.05,150,0.05)
sigma = 1.4e-4
f = 1.19e-4

ustar = 0.0126
zo = 0.0009


def planetary_bbl_structure(ustar,zo,z,f):
    '''
    Calculate vertical structure of a planetary boundary layer (a.k.a.
    bottom Ekman layer) following Soulsby (1983). The calculation assumes
    that the eddy viscosity increases linearly with height.
    '''

    i = 1j 
    kappa = 0.41 # Von Karman's constant
    
    # non-dimensional height and roughness length
    xi = 2*np.sqrt(f*z/(kappa*ustar))
    xio = 2*np.sqrt(f*zo/(kappa*ustar))
    
    # Q/Qinf in Equation 31 of Soulsby (1983)
    fac = (1-(ker(xi)*ker(xio) + kei(xi)*kei(xio))/
             (ker(xio)**2 + kei(xio)**2) + 
           i*(ker(xi)*kei(xio) - kei(xi)*ker(xio))/
             (ker(xio)**2 + kei(xio)**2))

    return fac

### Soulsby test cases

#### Case 1: Rectilinear free stream flow

In [12]:
Rpinf = 0.15
Rminf = 0.15

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

plt.figure()
plt.subplot(121)
plt.plot(np.abs(Rp)/Rpinf,z)
plt.plot(np.abs(Rm)/Rpinf,z)
plt.ylim([0,150])
plt.xlim([0,1.05])
plt.xlabel('normalized amplitude')
plt.ylabel('height [m]')
plt.legend(['AC','CW'])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

plt.subplot(122)
plt.plot(phip*180/np.pi,z)
plt.plot(-phim*180/np.pi,z)
plt.ylim([0,150])
plt.xlim([-0.5,10])
plt.xlabel('phase')
plt.legend(['AC','CW'])
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

FigureCanvasNbAgg()

In [13]:
plt.figure()
ax1 = plt.subplot(121)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
plt.ylabel('height [m]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)

plt.subplot(122)
plt.plot(phi*180/np.pi,z)
plt.plot(PHI*180/np.pi,z)
plt.ylim([0,150])
plt.legend(['$\phi$','$\Phi$'])
plt.xlabel('$\phi$, $\Phi$ [degrees]')

FigureCanvasNbAgg()

Text(0.5, 0, '$\\phi$, $\\Phi$ [degrees]')

#### Cases 2 and 3: Anticlockwise and clockwise free stream ellipses

In [14]:
Rpinf = 0.75*0.3
Rminf = 0.25*0.3

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

plt.figure()
ax1 = plt.subplot(121)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
plt.ylabel('height [m]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)

FigureCanvasNbAgg()

In [114]:
Rpinf = 0.25*0.3
Rminf = 0.75*0.3

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = -np.angle(Rm)

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

ax1 = plt.subplot(122)
ax1.plot(Ua,z)
plt.xlim([0,0.3])
plt.ylim([0,150])
plt.xlabel('Ua [m/s]')
ax2 = ax1.twiny()
ax2.plot(Ub/Ua,z)
plt.xlim([0,-0.6])
plt.ylim([0,150])
plt.xlabel('Ub/Ua')
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


### Station M 

Manually tune parameters to Station M data. In the future, parameters will be found through nonlinear optimization.

In [115]:
print(ap[zi[-1],m2i])
print(am[zi[-1],m2i])

<xarray.DataArray (constituent: 1)>
array([-0.012723-0.010618j])
Coordinates:
    height       float64 17.6
  * constituent  (constituent) object 'M2'
<xarray.DataArray (constituent: 1)>
array([0.016839+0.003567j])
Coordinates:
    height       float64 17.6
  * constituent  (constituent) object 'M2'


In [214]:
Rpinf = -0.012723-0.010618j
Rminf = 0.016839+0.003567j

if sigma+f < 0:
    Rpinf = np.abs(Rpinf)*np.exp(-i*np.angle(Rpinf))
if sigma-f > 0:
    Rminf = np.abs(Rminf)*np.exp(-i*np.angle(Rminf))

z = np.arange(0.05,25,0.05)
sigma = 1.4e-4
f = gsw.f(35+8.4585/60)
ustar = 0.002
zo = 0.02

Rp = Rpinf*planetary_bbl_structure(ustar,zo,z,sigma+f)
Rm = Rminf*planetary_bbl_structure(ustar,zo,z,sigma-f)

phip = np.angle(Rp)
phim = np.angle(Rm)

if sigma+f < 0:
    # southern hemisphere (needs testing) 
    # tidal oscillation lower frequency than f
    phip = -np.angle(Rp)
    Rp = np.abs(Rp)*np.exp(1j*phip)
    
if sigma-f > 0:
    # northern hemisphere 
    # tidal oscillation higher frequency than f
    phim = -np.angle(Rm)
    Rm = np.abs(Rm)*np.exp(1j*phim)
    

phi = (phim-phip)/2
PHI = (phim+phip)/2

Ua = np.abs(Rp) + np.abs(Rm)
Ub = np.abs(Rp) - np.abs(Rm)

plt.figure()
plt.plot(Ap[:,m2i][zi],dstide['height'][zi],'rs')
plt.plot(Am[:,m2i][zi],dstide['height'][zi],'bs')
plt.plot(np.abs(Rp),z,'r')
plt.plot(np.abs(Rm),z,'b')
plt.legend(['anticlockwise (+)','clockwise (-)'])
plt.title('magnitude of rotary coefficients')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'magnitude of rotary coefficients')

In [215]:
plt.figure()
plt.plot(dstide['g'][:,m2i][zi],dstide['height'][zi],'rs')
plt.plot(dstide['theta'][:,m2i][zi],dstide['height'][zi],'bs')
# add 180 degrees?
plt.plot(phi*180/np.pi+180,z,'r')
plt.plot(PHI*180/np.pi+180,z,'b')
plt.legend(['$g$, $\phi$','$\\theta$, $\Phi$'])
plt.title('phase of rotary coefficients')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'phase of rotary coefficients')

In [216]:
plt.figure()
plt.plot(np.real(ap[:,m2i][zi]),dstide['height'][zi],'rs')
plt.plot(np.imag(ap[:,m2i][zi]),dstide['height'][zi],'ro')
plt.plot(np.real(am[:,m2i][zi]),dstide['height'][zi],'bs')
plt.plot(np.imag(am[:,m2i][zi]),dstide['height'][zi],'bo')
plt.plot(np.real(Rp),z,'r-')
plt.plot(np.imag(Rp),z,'r--')
plt.plot(np.real(Rm),z,'b-')
plt.plot(np.imag(Rm),z,'b--')
plt.title('complex rotary coefficients')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'complex rotary coefficients')

That looks about right!

In a more objective procedure, the model should be fit to these complex rotary coefficients. The free parameters for such a fit might include:

* Anticlockwise coefficients above BBL: $\text{Re}(R_+^{\infty})$, $\text{Im}(R_+^{\infty})$
* Clockwise coefficients above BBL: $\text{Re}(R_-^{\infty})$, $\text{Im}(R_-^{\infty})$
* Maximum shear velocity ($u_{*m}$)
* Roughness length ($z_o$)

This is a total of six parameters. The coefficients above the BBL could also potentially be treated as known paramaters, using the top bin from the data.

The M2 constituent is the strongest signal. Can other constituents be used? Will the value of $z_o$ change?